In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()
num_examples = 10
run_dir = "Data"
hemisphere_symmetric = 1
random_seed = 12345
random_sampling = 0

# Generate and Save Training Data

In [ ]:
# Warning to save individual illumination data change "sys_params["run_clean"]" to False in "training_data_generation.py"
dataset_params, sys_params, facility_spec = tdg.main((_, run_dir, num_examples, hemisphere_symmetric, random_sampling, random_seed))

# Diagnostic

In [ ]:
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import os
%matplotlib inline
np_complex = np.vectorize(complex)

In [ ]:
diag_dir = "Data"
iex = 0
with_pointing_markers = True
import_flipped = False
old_format = False
display_steradians = False
sys_params = tdg.define_system_params(diag_dir)

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)

In [ ]:
ind_profile = 0
dataset_params = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["dataset_params_filename"])
facility_spec = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["facility_spec_filename"])
dataset = nrw.read_general_netcdf(sys_params["root_dir"]+"/"+sys_params["trainingdata_filename"])

avg_flux = dataset["avg_flux"][iex, ind_profile]
real_modes = dataset["real_modes"][iex,ind_profile,:]
imag_modes = dataset["imag_modes"][iex,ind_profile,:]
rms = dataset["rms"][iex, ind_profile]
#rms = uim.alms2rms(real_modes, imag_modes, dataset_params["LMAX"])

print('Intensity per steradian, {:.2e}W/sr'.format(avg_flux))
print("The LLE quoted rms cumulative over all modes is: ", rms*100.0, "%")

intensity_map_normalized = uim.modes2imap(real_modes, imag_modes, dataset_params["imap_nside"])
intensity_map_sr = (intensity_map_normalized+1)*avg_flux

if display_steradians:
    hp.mollview(intensity_map_sr, unit=r"$\rm{W/sr}$",flip="geo")
else:
    hp.mollview((intensity_map_normalized+1)*avg_flux / (facility_spec['target_radius'] / 10000.0)**2, unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()

complex_modes = np_complex(real_modes, imag_modes)
power_spectrum = uim.alms2power_spectrum(complex_modes, dataset_params["LMAX"])
print("The LLE quoted rms cumulative over all modes is: ", np.sqrt(np.sum(power_spectrum))*100.0, "%")

LMAX = dataset_params["LMAX"]
fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), np.sqrt(power_spectrum) * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, LMAX])
plt.title("Unweighted Modes Python")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/modes_"+str(iex) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
if with_pointing_markers:
    if display_steradians:
        hp.mollview((intensity_map_normalized+1)*avg_flux, unit=r"$\rm{W/sr}$",flip="geo")
    else:
        hp.mollview((intensity_map_normalized+1)*avg_flux / (facility_spec['target_radius'] / 10000.0)**2, unit=r"$\rm{W/cm^2}$",flip="geo")
    hp.graticule()
    
    deck_gen_params = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["deck_gen_params_filename"])
    print_list1, mean_power_fraction = uim.extract_run_parameters(iex, dataset_params, facility_spec, sys_params, deck_gen_params)
    print_list2 = uim.readout_intensity(facility_spec, intensity_map_sr, mean_power_fraction=mean_power_fraction)

    print_list = print_list1 + print_list2
    stats_filename = sys_params["figure_location"]+"/stats"+str(iex)+".txt"
    if os.path.exists(stats_filename):
        os.remove(stats_filename)
    file1 = open(stats_filename,"a")
    for line in range(len(print_list)):
        print(print_list[line])
        file1.writelines(print_list[line]+"\n")
    file1.close()
    
    port_theta = deck_gen_params["port_centre_theta"]
    port_phi = deck_gen_params["port_centre_phi"]
    hp.projscatter(port_theta, port_phi)
    
    pointing_theta = np.squeeze(deck_gen_params["theta_pointings"][iex,:])
    pointing_phi = np.squeeze(deck_gen_params["phi_pointings"][iex,:])
    hp.projscatter(pointing_theta, pointing_phi)
    
    if (facility_spec['facility'] == "NIF"):
        num_ifriit_beams = int(facility_spec['nbeams'] / facility_spec['beams_per_ifriit_beam'])
        skip = int(facility_spec['nbeams'] / facility_spec['num_quads'])
    elif (facility_spec['facility'] == "LMJ"):
        num_ifriit_beams = int(facility_spec['nbeams'] / facility_spec['beams_per_ifriit_beam'])
        skip = 1
    
    for iport in range(0, num_ifriit_beams, skip):
        x, y = uim.angle2moll(port_theta[iport], port_phi[iport])
        x2, y2 = uim.angle2moll(pointing_theta[iport], pointing_phi[iport])
        dx = x2 - x
        dy = y2 - y

        # reduce line length to give space for arrow head
        head_length = 0.05
        shaft_angle = np.arctan2(dy, dx)
        shaft_length = np.sqrt(dx**2 + dy**2) - head_length
        if shaft_length > 0.0:
            dx = shaft_length * np.cos(shaft_angle)
            dy = shaft_length * np.sin(shaft_angle)
            
            if np.abs(dx) < 1.0: # don't plot arrow for pointings which cross the edge
                plt.arrow(x, y, dx, dy, head_width=0.05, head_length=head_length, fc='k', ec='k')
        else:
            plt.arrow(x, y, dx, dy)
    plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_'+str(iex) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Analysis from ifriit output

In [ ]:
filename_flipped_trainingdata = "flipped_training_data_and_labels.nc"

In [ ]:
num_examples = 1
sys_params = tdg.define_system_params(diag_dir)
sys_params["run_gen_deck"] = False

if old_format:
    dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
else:
    dataset_params = nrw.read_general_netcdf(diag_dir + "/" + sys_params["dataset_params_filename"])
    facility_spec = nrw.read_general_netcdf(diag_dir + "/" + sys_params["facility_spec_filename"])
dataset_params["num_examples"] = num_examples

In [ ]:
if old_format:
    intensity_map_sr = nrw.read_intensity(run_location, dataset_params["imap_nside"], facility_spec['target_radius'])
    # convert from W/sr to W/cm**2
    intensity_map = intensity_map_sr / (facility_spec['target_radius'] / 10000.0)**2
else:
    parameters = nrw.read_general_netcdf(run_location + "/" + sys_params["ifriit_ouput_name"])
    intensity_map = parameters["intensity"]
    # convert from W/cm**2 to W/sr
    intensity_map_sr = parameters["intensity"] * (facility_spec['target_radius'] / 10000.0)**2

if display_steradians:
    hp.mollview(intensity_map_sr, unit=r"$\rm{W/sr}$", flip="geo")
else:
    hp.mollview(intensity_map, unit=r"$\rm{W/cm^2}$", flip="geo")
hp.graticule()
plt.savefig(run_location+"/init_illumination" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted = uim.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(run_location+"/init_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
hp.mollview((intensity_map_normalized+1)*avg_power, unit=r"$\rm{W/cm^2}$", flip="geo")

real_modes, imag_modes = uim.imap2modes(intensity_map_normalized, LMAX, avg_power)
intensity_map_normalized2 = uim.modes2imap(real_modes, imag_modes, dataset_params["imap_nside"])
hp.mollview((intensity_map_normalized2+1)*avg_power, unit=r"$\rm{W/cm^2}$", flip="geo")

## Open Ifriit heat source modes

In [ ]:
file_name = sys_params["heat_source_nc"]

hs_and_modes = nrw.read_general_netcdf(run_location+"/"+file_name)

avg_flux = hs_and_modes["average_flux"][0]
print('Intensity per steradian, {:.2e}W/sr'.format(avg_flux))

lmax = 30
np_complex = np.vectorize(complex)
X_train_complex = np_complex(hs_and_modes["complex_modes"][0,:], hs_and_modes["complex_modes"][1,:])

intensity_map_normalized = hp.alm2map(X_train_complex, lmax)
#hp.mollview(intensity_map_normalized, unit=r"$\rm{W/sr}$",flip="geo")
hp.mollview((intensity_map_normalized+1)*avg_flux, unit=r"$\rm{W/sr}$",flip="geo")

hp.mollview((intensity_map_normalized+1)*avg_flux / (facility_spec['target_radius'] / 10000.0)**2, unit=r"$\rm{W/cm^2}$",flip="geo")

theta_edges = (hs_and_modes["theta"][1:] + hs_and_modes["theta"][:-1]) / 2.0
theta_edges = np.append(0.0, theta_edges)
theta_edges = np.append(theta_edges, np.pi)
phi_edges = (hs_and_modes["phi"][1:] + hs_and_modes["phi"][:-1]) / 2.0
phi_edges = np.append(0.0, phi_edges)
phi_edges = np.append(phi_edges, 2.0*np.pi)

theta_grid, phi_grid = np.meshgrid(theta_edges, phi_edges)

dphi = phi_grid[1:,1:] - phi_grid[:-1,1:]
d_cos_theta = np.cos(theta_grid[1:,1:]) - np.cos(theta_grid[1:,:-1])
domega = np.abs(dphi * d_cos_theta)

total_pwr = np.sum(hs_and_modes["heat_source"] * domega)
surface_area = 4.0 * np.pi * (facility_spec['target_radius'] / 10000.0)**2
avg_flux = total_pwr / (4.0 * np.pi)

print('Mean intensity is {:.2e}W/cm^2, '.format(total_pwr / surface_area)) #/ 4.0 / np.pi
print('Intensity per steradian, {:.2e}W/sr'.format(avg_flux))

power_spectrum = uim.alms2power_spectrum(X_train_complex, lmax)

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), np.sqrt(power_spectrum) * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, LMAX])
plt.title("Unweighted Modes Python")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")

In [ ]:
theta_grid, phi_grid = np.meshgrid(hs_and_modes["theta"], hs_and_modes["phi"])
plt.pcolor(theta_grid, phi_grid, hs_and_modes["heat_source"])
plt.colorbar()

nside_low = 16
nside = 32

pixel_ind = hp.ang2pix(nside_low, theta_grid.reshape(-1), phi_grid.reshape(-1))
imap_low = np.zeros(hp.nside2npix(nside_low))
imap_low[pixel_ind] = hs_and_modes["heat_source"].reshape(-1)
imap_upgrade = hp.ud_grade(imap_low, nside)

pix_ind = hp.ang2pix(nside, theta_grid.reshape(-1), phi_grid.reshape(-1))
imap_high = np.zeros(hp.nside2npix(nside))
imap_upgrade[pix_ind] = hs_and_modes["heat_source"].reshape(-1)

# convert from W/sr to W/cm**2  
hp.mollview(imap_upgrade / (facility_spec['target_radius'] / 10000.0)**2, unit=r"$\rm{W/cm^2}$",flip="geo")
print('Mean intensity is {:.2e}W/cm^2, '.format(np.mean(imap_upgrade) / (facility_spec['target_radius'] / 10000.0)**2))
hp.graticule()
plt.savefig(run_location+"/heatsource" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

LMAX = 30
power_spectrum_unweighted = uim.power_spectrum(imap_upgrade, LMAX)

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, LMAX])
plt.title("Unweighted Modes Python")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(run_location+"/hs_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Check aster input netcdf

In [ ]:
from netCDF4 import Dataset

In [ ]:
def read_general_netcdf_test(filename):
    parameters = {}

    rootgrp = Dataset(filename + ".nc")
     
    grp = rootgrp["data"]
    keys = list(grp.variables.keys())
    for key in keys:
        if np.shape(np.shape(grp[key]))[0] == 2:
            parameters[key] = grp[key][:,:]
        if np.shape(np.shape(grp[key]))[0] == 1:
            parameters[key] = grp[key][:]
    rootgrp.close()

    return parameters

In [ ]:
file_name = "ifriit_1davg_input.nc" #"N210519" #"N190204-003"
aster_input = nrw.read_general_netcdf(run_location+"/"+file_name)
print(aster_input.keys())

"""
run_loc2 = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(0) #change comparison file here
file_name = "ifriit_1davg_input.nc" #"N210519" #"N190204-003"
aster_input2 = nrw.read_general_netcdf(run_loc2+"/"+file_name)

for key in aster_input.keys():
    print(np.shape(aster_input[key]), np.shape(aster_input[key]))
    #print(aster_input[key], aster_input2[key])
#"""

plt.figure()
var = "ne"
plt.plot(aster_input['xs'].reshape(-1),np.log10(aster_input[var].reshape(-1)),label="a")
#plt.plot(aster_input2['xs'].reshape(-1),np.log10(aster_input2[var].reshape(-1)),label="b")
plt.xlabel(r"Radius ($\mu m$)")
plt.ylabel(var)
plt.legend()
plt.savefig(run_location+"/profiles_" + var + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

# Combine Datasets

In [ ]:
from netCDF4 import Dataset
from os import path
import os
import numpy as np
import netcdf_read_write as nrw
sys_params = tdg.define_system_params(run_dir)

In [ ]:
dataset1 = "Data1"
dataset2 = "Data2"

new_file = "Data3"

filename_trainingdata = dataset1 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_1 = training_data.variables["X_train"][:]
Y_1 = training_data.variables["Y_train"][:]
avg_powers_1 = training_data.variables["avg_powers"][:]
training_data.close()

filename_trainingdata = dataset2 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_2 = training_data.variables["X_train"][:]
Y_2 = training_data.variables["Y_train"][:]
avg_powers_2 = training_data.variables["avg_powers"][:]
training_data.close()

X_train = np.hstack((X_1, X_2))
Y_train = np.hstack((Y_1, Y_2))
avg_powers = np.hstack((avg_powers_1, avg_powers_2))
print(np.shape(X_train), np.shape(Y_train), np.shape(avg_powers))

filename_trainingdata = new_file + "/" + sys_params["trainingdata_filename"] 
nrw.save_training_data(X_train, Y_train, avg_powers, filename_trainingdata)

# Create time varying input deck

In [ ]:
run_dir = "Data_min10"
iex = 0
num_examples = 1
import_flipped = True

sys_params = tdg.define_system_params(run_dir)
sys_params["run_gen_deck"] = False
if import_flipped:
    sys_params["trainingdata_filename"] = filename_flipped_trainingdata
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))

X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)
dataset_params["Y_train"] = Y_all[:,iex]
if import_flipped:
    dataset_params["Y_train"] = X_all[:,iex]
print(np.shape(dataset_params["Y_train"]))
print(dataset_params["Y_train"])
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

In [ ]:
num_cones = int(facility_spec['num_cones']/2) # Symmetry in top bottom hemisphere
num_params = int(dataset_params["num_variables_per_beam"] * facility_spec['num_cones']/2)

power_slice = slice(3, num_params, num_cones)
cone_powers = dataset_params["Y_train"][power_slice] / 2.0 + dataset_params["min_power"]

print(cone_powers)

total_power = 0.0
for icone in range(num_cones):
    beams_per_cone = facility_spec['beams_per_cone'][icone]
    total_power += cone_powers[icone] * beams_per_cone * 2.0
mean_power_fraction = total_power / facility_spec['nbeams']

print(mean_power_fraction)

In [ ]:
import csv
import utils_intensity_map as uim

The laser pulse selected is from N210519. The powers are different per ring so we find the total power.

In [ ]:
file_name = "laser_pulse_data/I_PDD_BePDD_DDShap_EEE_Ring"
file_type = ".csv"

num_lines = sum(1 for line in open(file_name + str(1) + file_type))

times = np.zeros((num_cones,num_lines))
powers = np.zeros((num_cones,num_lines))
fig = plt.figure()
ax = plt.axes()

for icone in range(num_cones):
    pulse_file = open(file_name + str(icone+1) + file_type)
    pulse_data = csv.reader(pulse_file)
    i = 0
    for row in pulse_data:
        times[icone,i] = float(row[0])
        powers[icone,i] = float(row[1])
        i += 1

    pulse_file.close()
    plt.plot(times[icone,:], powers[icone,:])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
    
total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")

In [ ]:
run_dir = "Data_time_varying"
run_location = run_dir + "/run_0/"
laser_init_time = 0.0

Y_save = dataset_params["Y_train"]
dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
if hemisphere_symmetric == 0:
    dataset_params["hemisphere_symmetric"] = False
dataset_params["Y_train"] = Y_save
print(dataset_params["Y_train"])

avg_single_beam = total_power / facility_spec['nbeams'] / mean_power_fraction

cone_multiplier = 0.0
for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = run_location + "pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        cone_multiplier = cone_powers[0]
    elif (cone_name == 30):
        cone_multiplier = cone_powers[1]
    elif (cone_name == 44.5):
        cone_multiplier = cone_powers[2]
    else:
        cone_multiplier = cone_powers[3]
    
    with open(file_name,"w") as out:
        for irow in range(len(avg_single_beam)):
            out.write(str(times[0,irow]) + ' ' + str(avg_single_beam[irow] * cone_multiplier))
            out.write('\r\n')
    out.close()
    
fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], avg_single_beam * cone_powers[0])
plt.plot(times[0,:], avg_single_beam * cone_powers[1])
plt.plot(times[0,:], avg_single_beam * cone_powers[2])
plt.plot(times[0,:], avg_single_beam * cone_powers[3])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")

sys_params = tdg.define_system_params(run_dir)
sys_params["run_clean"] = False
facility_spec['t0'] = laser_init_time

dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

idg.generate_input_deck(facility_spec, run_location)
idg.generate_input_pointing_and_pulses(facility_spec, run_location, dataset_params["run_type"])

# Check Polar Coord Generation

In [ ]:
import healpy_pointings as hpoint

In [ ]:
nr = 10000

x = np.random.rand(nr) * 2.0 - 1.0
y = np.random.rand(nr) * 2.0 - 1.0
theta_max = np.pi / 4.0

cart_x_coords = np.zeros((nr))
cart_y_coords = np.zeros((nr))
polar_p_coords = np.zeros((nr))
polar_t_coords = np.zeros((nr))
for ir in range(nr):
    cart_x_coords[ir] = x[ir]
    cart_y_coords[ir] = y[ir]

    r, offset_phi = hpoint.square2disk(x[ir], y[ir])
    offset_theta = theta_max * r

    polar_p_coords[ir] = offset_phi
    polar_t_coords[ir] = offset_theta

X = cart_x_coords
Y = cart_y_coords

fig = plt.figure()
plt.plot(X,Y,"*")

print(np.min(polar_p_coords))
print(np.max(polar_p_coords))

U = np.degrees(polar_t_coords * np.cos(polar_p_coords))
V = np.degrees(polar_t_coords * np.sin(polar_p_coords))

fig1 = plt.figure()
plt.plot(U,V,"*")

In [ ]:
nx = 50
ny = 50

x = np.linspace(-1, 1, nx)
y = np.linspace(-1, 1, ny)
theta_max = np.pi / 4.0

cart_x_coords = np.zeros((nx,ny))
cart_y_coords = np.zeros((nx,ny))
polar_p_coords = np.zeros((nx,ny))
polar_t_coords = np.zeros((nx,ny))
for ix in range(nx):
    for iy in range(ny):
        cart_x_coords[ix,iy] = x[ix]
        cart_y_coords[ix,iy] = y[iy]
        
        r, offset_phi = hpoint.square2disk(x[ix], y[iy])
        offset_theta = theta_max * r
        
        polar_p_coords[ix,iy] = offset_phi
        polar_t_coords[ix,iy] = offset_theta

X = cart_x_coords.flatten()
Y = cart_y_coords.flatten()

fig = plt.figure()
plt.plot(X,Y,"*")

phi = polar_p_coords.flatten()
theta = polar_t_coords.flatten()
U = np.degrees(theta * np.cos(phi))
V = np.degrees(theta * np.sin(phi))

fig1 = plt.figure()
plt.plot(U,V,"*")

# Neural Network

In [ ]:
import neural_network_generation as nng
num_epochs = 20
num_nn = 1
use_final_sigmoid = 1

In [ ]:
nn_params, nn_dataset, sys_params, nn_hyperparams = nng.main((_, run_dir, num_epochs, num_nn, use_final_sigmoid))

In [ ]:
fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Neural Network Random Seed')

## Compare NN

In [ ]:
import netcdf_read_write as nrw

In [ ]:
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Random Seed')

## Compare NN accuracy from weights

In [ ]:
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import neural_network_generation as nng

In [ ]:
num_nn = 1

In [ ]:
initialize_seed = np.arange(num_nn)
test_acc = np.zeros(num_nn)

sys_params = tdg.define_system_params(run_dir)
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

nn_params = nng.define_nn_params(num_nn)
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

for inn in range(num_nn):
    nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
    nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])
    nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
    y_pred = tfnn.apply_network(nn_dataset["X_test"], nn_weights, nn_hyperparams["use_final_sigmoid"][inn])
    test_acc[inn] = np.mean(np.abs(nn_dataset["Y_test"] - y_pred.T))

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(initialize_seed, test_acc, "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Neural Network Random Seed')
plt.savefig(sys_params["figure_location"]+"/Compare_NN" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Run Prediction Through Ifriit

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
%matplotlib inline
num_epochs = 10

In [ ]:
run_dir = "Data"
test_pred_dir = "Data_test_pred_1"

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# choose test data set
sys_params = tdg.define_system_params(run_dir)
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)
nn_params = nng.define_nn_params(num_nn)
nn_params["use_test_set"] = False
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)

nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])
dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
LMAX = dataset_params["LMAX"]
dataset_params["Y_train"] = nn_dataset["Y_train"].T
dataset_params["X_train"] = nn_dataset["X_train"].T
ifriit_inputs_true = Y_all
modes_true_all = X_all

# Change the number of modes to LMAX
modes_true = nng.change_number_modes(modes_true_all, avg_powers_all, LMAX)

# apply neural net to test set
sys_params = tdg.define_system_params(test_pred_dir)
ifriit_inputs_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights, nn_hyperparams["use_final_sigmoid"][inn])
dataset_params["Y_train"] = ifriit_inputs_pred

mean_true_val = np.mean(ifriit_inputs_true)
print("The absolute error is: ", np.mean(np.abs(ifriit_inputs_true - ifriit_inputs_pred)))
print("The percentage error relative to the mean is: ", np.mean((np.abs(ifriit_inputs_true - ifriit_inputs_pred))) / mean_true_val * 100, "%")

abs_error = np.mean(np.abs(ifriit_inputs_true - ifriit_inputs_pred), axis=1)
rel_error = np.mean(np.abs(ifriit_inputs_true - ifriit_inputs_pred)/ ifriit_inputs_true , axis=1) * 100
print(np.shape(abs_error))
print(abs_error)
print(rel_error)

for icone in range(4):
    print("Cone ", icone+1, " error: {:.2f}%".format(np.sum(abs_error[slice(icone,16,4)]) / 4.0 * 100.0 / mean_true_val))

parameter_names = ["offset_theta", "offset_phi", "cone_defocus", "cone_power"]
for icone in range(4):
    print(parameter_names[icone], " error: {:.2f}%".format(np.sum(abs_error[icone*4:(icone+1)*4]) / 4.0 * 100.0 / mean_true_val))

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
sys_params = tdg.define_system_params(test_pred_dir)
num_nn = 1
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)
modes_pred = uim.change_number_modes(X_all, avg_powers_all, LMAX)

mean_val_true = np.mean(np.abs(modes_true))
print("The absolute error is: ", np.mean(np.abs(modes_true - modes_pred)))
print("Guessing zero for all modes gives error: ", np.mean(np.abs(modes_true)))
print("The percentage relative error is: ", np.mean(np.abs(modes_true - modes_pred) / modes_true) * 100.0, "%")

abs_error = np.mean(np.abs(modes_true - modes_pred), axis=1)
rel_error = np.mean(np.abs(modes_true - modes_pred) / modes_true, axis=1) * 100.0
print(np.shape(abs_error))
print(abs_error)
print(rel_error)

fig = plt.figure()
ax = plt.axes()
plt.plot(abs_error)
plt.xlabel("Mode")
plt.ylabel("Average Absolute Error")
plt.savefig(sys_params["figure_location"]+'/absolute_error_averaged_'+ str(nn_params["num_examples"]) +"ex" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Automate Finding Minima

In [ ]:
test_pred_dir = "Data_minima3"

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# choose test data set
sys_params = tdg.define_system_params("Data_220713_100ex")
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)
nn_params = nng.define_nn_params(num_nn)
nn_params["use_test_set"] = False
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)

nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])
#print(nn_dataset["X_train"][1,0], nn_hyperparams["mu"][inn]/nn_hyperparams["sigma"][inn], nn_hyperparams["mu"][inn]-nn_hyperparams["sigma"][inn])
#print(np.shape(nn_dataset["X_train"]))
#print(nn_dataset["X_train"][0,:])
new_min = nn_hyperparams["mu"][inn]/nn_hyperparams["sigma"][inn]
nn_dataset["X_train"] = - new_min + new_min * np.random.rand(np.shape(nn_dataset["X_train"])[0], np.shape(nn_dataset["X_train"])[1])
#print(np.shape(nn_dataset["X_train"]))
#print(nn_dataset["X_train"][0,:])
dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
dataset_params["Y_train"] = nn_dataset["Y_train"].T
y_true = dataset_params["Y_train"]

# apply neural net to test set
sys_params = tdg.define_system_params(test_pred_dir)
y_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights, nn_hyperparams["use_final_sigmoid"][inn])
dataset_params["Y_train"] = y_pred

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
iex = 0

num_examples = 1
sys_params = tdg.define_system_params(test_pred_dir)
sys_params["run_gen_deck"] = False
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
dataset_params["Y_train"] = Y_all[:,iex]
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

mean_power_fraction = uim.extract_run_parameters(dataset_params, facility_spec, sys_params)

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"], run_data['target_radius'])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)


# convert from W/sr to W/cm**2
intensity_map = intensity_map * (facility_spec['target_radius'] / 10000.0)**2
hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(np.squeeze(dataset_params["theta_pointings"]), np.squeeze(dataset_params["phi_pointings"]))
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_'+ str(iex) + "low_rms" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+ str(iex) +"low_rms" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Run PDD through NN

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
import utils_intensity_map as uim
%matplotlib inline
num_epochs = 10
num_examples = 1
nif_pdd_dir = "Data_pdd"
nif_pdd_pred_dir = "Data_pdd_pred"

In [ ]:
imap_nside = 256
LMAX = 30
input_size = 992
output_size = 16
target_radius_microns = 1100

run_location = nif_pdd_dir + "/run_0"
intensity_map = nrw.read_intensity(run_location, imap_nside, target_radius_microns)
X_train, avg_powers_all = uim.create_xtrain(intensity_map, LMAX)
X_all = np.reshape(X_train, (-1,input_size))
Y_all = np.zeros(output_size)

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights("neural_network_weights" + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# param setup
sys_params = tdg.define_system_params(nif_pdd_pred_dir)
nn_params["use_test_set"] = False
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])

y_pred = tfnn.apply_network(nn_dataset["X_train"].T, nn_weights, nn_hyperparams["use_final_sigmoid"][inn])

dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
dataset_params["Y_train"] = y_pred

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

# NN as surrogate for Ifriit

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
%matplotlib inline
num_epochs = 10
num_nn = 1
run_dir = "Data"
imap_nside = 256
LMAX = 30
filename_flipped_trainingdata = "flipped_training_data_and_labels.nc"
use_final_sigmoid = 0

In [ ]:
# choose test data set
sys_params = tdg.define_system_params(run_dir)
nn_params = nng.define_nn_params(num_nn)

X_all, Y_all, avg_powers_all = nrw.import_training_data_reversed(sys_params, LMAX)
filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata 
nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)

In [ ]:
sys_params = tdg.define_system_params(run_dir)
sys_params["trainingdata_filename"] = filename_flipped_trainingdata
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

In [ ]:
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
#nn_dataset = nng.normalise(nn_dataset)
nn_hyperparams = nng.define_nn_hyperparams(num_epochs, num_nn, use_final_sigmoid=use_final_sigmoid)
nn_hyperparams = nng.multiple_nn(nn_params, nn_dataset, sys_params, nn_hyperparams)

## Check performance on test set

In [ ]:
use_final_sigmoid = 0
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
y_pred = tfnn.apply_network(nn_dataset["X_test"], nn_weights, use_final_sigmoid)

In [ ]:
iex = 0

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), nn_dataset["Y_test"][iex,:] * 100.0)
#plt.plot(np.arange(LMAX), Y_all[:,iex])
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, Ifriit Inverse Ray Trace")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"original" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,iex] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate Model")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Test on Independent Set

In [ ]:
run_dir = "Data_220713_100ex"
filename_flipped_trainingdata = "flipped_training_data_and_labels.nc"

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# choose test data set
sys_params = tdg.define_system_params(run_dir)
sys_params["trainingdata_filename"] = filename_flipped_trainingdata
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

nn_params = nng.define_nn_params(num_nn)
nn_params["use_test_set"] = False
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params, use_test_set=False)

dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
dataset_params["Y_train"] = nn_dataset["Y_train"].T
dataset_params["X_train"] = nn_dataset["X_train"].T
ifriit_inputs_true = dataset_params["X_train"]
modes_true = dataset_params["Y_train"]

# apply neural net to test set
modes_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights, nn_hyperparams["use_final_sigmoid"][inn])

In [ ]:
mean_val_true = np.mean(np.abs(modes_true))
print("The absolute error is: ", np.mean(np.abs(modes_true - modes_pred)))
print("Guessing zero for all modes gives error: ", np.mean(np.abs(modes_true)))
print("The percentage relative error is: ", np.mean(np.abs(modes_true - modes_pred) / modes_true) * 100.0, "%")

abs_error = np.mean(np.abs(modes_true - modes_pred), axis=1)
rel_error = np.mean(np.abs(modes_true - modes_pred) / modes_true, axis=1) * 100.0
print(np.shape(abs_error))
print(abs_error)
print(rel_error)

fig = plt.figure()
ax = plt.axes()
plt.plot(abs_error)
plt.xlabel("Mode")
plt.ylabel("Average Absolute Error")
plt.savefig(sys_params["figure_location"]+'/absolute_error_averaged_'+ str(nn_params["num_examples"]) + "ex" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Find Minimum in Train set

In [ ]:
use_final_sigmoid = 0
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
y_pred = tfnn.apply_network(X_all.T, nn_weights, use_final_sigmoid)

In [ ]:
mindex = np.argmin(np.mean(Y_all, axis=0))
print(mindex, " out of ", np.shape(Y_all))
mindex = np.argmin(np.sqrt(np.sum(Y_all**2, axis=0)))
print(mindex, " out of ", np.shape(Y_all))
print(np.sqrt(np.sum(Y_all[:,mindex]**2, axis=0)))
print(np.shape(Y_all[:,mindex]), X_all[:,mindex])

In [ ]:
fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), Y_all[:,mindex] * 100.0)
#plt.plot(np.arange(LMAX), Y_all[:,iex])
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, Ifriit Inverse Ray Trace")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(mindex)+"original" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,mindex] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate Model")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(mindex)+"nn" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
num_examples = 10
run_dir = "Data_min10"

#min_slice = np.argsort(np.mean(Y_all, axis=0))[:num_examples]
min_slice = np.argsort(np.sqrt(np.sum(Y_all**2, axis=0)))[:num_examples]
print(min_slice)

dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
dataset_params["Y_train"] = X_all[:,min_slice]

In [ ]:
sys_params = tdg.define_system_params(run_dir)

# Create new run files
#sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
#facility_spec['target_radius'] = 1053.0
#facility_spec['default_power'] = 2.5

In [ ]:
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

## Search for Minima

In [ ]:
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
x_test_random = np.random.rand(1000000,16)
print(x_test_random[0,:])

In [ ]:
use_final_sigmoid = 0
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
#y_pred = tfnn.apply_network(X_all.T, nn_weights, use_final_sigmoid)
y_pred = tfnn.apply_network(x_test_random, nn_weights, use_final_sigmoid)

In [ ]:
num_examples = 10
iex = 0
mindex = np.argmin(np.mean(y_pred, axis=0))
min_slice = np.argsort(np.mean(y_pred, axis=0))[:num_examples]
print(mindex)
print(np.shape(x_test_random[min_slice[:],:].T))

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,min_slice[iex]] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate find minimum")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
#print(np.shape(dataset_params["Y_train"]))
#print(np.shape(x_test_random[min_slice,:]))
dataset_params["Y_train"] = x_test_random[min_slice[:],:].T

run_dir = "Data_nn_find_min"
sys_params = tdg.define_system_params(run_dir)

# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
# choose test data set
sys_params = tdg.define_system_params(run_dir)

X_all, Y_all, avg_powers_all = nrw.import_training_data_reversed(sys_params, LMAX)
filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata 
nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)
dataset_params, facility_spec = tdg.define_dataset_params(np.shape(X_all)[1])

#mindex = np.argmin(np.mean(Y_all, axis=0))
mindex = np.argmin(np.sqrt(np.sum(Y_all**2, axis=0)))
print(mindex)

In [ ]:
iex = 0
with_pointing_markers = True

print(dataset_params[\"Y_train\"][iex])

num_examples = 1
sys_params = tdg.define_system_params(run_dir)
sys_params["run_gen_deck"] = False
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"], facility_spec['target_radius'])

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,min_slice[iex]] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate find Minimum")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn_min_guess" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"ifriit" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')